In [39]:
import sqlite3
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances
import math
import matplotlib.pyplot as plt
import warnings
import numpy as np
import pyodbc 
warnings.simplefilter('ignore')

In [40]:
DB = {'servername': 'LAPTOP-LPE28RPE\SQLEXPRESS', 
    'database': 'United_outdoors'}

export_conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + DB['servername'] + 
                              ';DATABASE=' + DB['database'])

export_cursor = export_conn.cursor()
export_cursor  

In [41]:

Adventure = {
    'servername' : 'LAPTOP-LPE28RPE\SQLEXPRESS',
    'database' : 'AdventureWorks2019'
}


Northwind = {
    'servername' : 'LAPTOP-LPE28RPE\SQLEXPRESS',
    'database' : 'Northwind'
}

access_db_path = r'C:\Users\Humberto de Castro\OneDrive\Desktop\SEM4\AenC\aenc.accdb'

#Connect to AdventureWorks
Adventure_conn = pyodbc.connect(f"DRIVER={{SQL Server}};SERVER={Adventure['servername']};DATABASE={Adventure['database']};Trusted_Connection=yes;")
Adventure_cursor = Adventure_conn.cursor()

#Connect to Northwind
Northwind_conn = pyodbc.connect(f"DRIVER={{SQL Server}};SERVER={Northwind['servername']};DATABASE={Northwind['database']};Trusted_Connection=yes;")
Northwind_cursor = Northwind_conn.cursor()

#Connect to AenC
AenC_conn = pyodbc.connect(f"DRIVER={{Microsoft Access Driver (*.mdb, *.accdb)}};DBQ={access_db_path};")



In [42]:
AdventureWorks_queries = {
    'AW_Customer' : 'SELECT * FROM Sales.Customer',
    'Person' : 'SELECT * FROM Person.Person',
    'BusinessEntity' : 'SELECT * FROM Person.BusinessEntity',
    'BusinessEntityAddress' : 'SELECT * FROM Person.BusinessEntityAddress',
    'PersonAddress' : 'SELECT * FROM Person.Address',
    'StateProvince' : 'SELECT * FROM Person.StateProvince',
    'CountryRegion' : 'SELECT * FROM Person.CountryRegion',
    'Territory' : 'SELECT * FROM Sales.SalesTerritory',
    'PersonPhone' : 'SELECT * FROM Person.PersonPhone',
    'Store' : 'SELECT * FROM Sales.Store'
}

Northwind_queries = {
    'NW_Customer' : 'SELECT * FROM Customers'
}

AenC_queries = {
    'state' : 'SELECT * FROM state',  # Add state table query
    'AC_Customer' : 'SELECT * FROM Customer',  # Add Customer table query
    'region' : 'SELECT * FROM region'
}

dataframes = {}

# Lees elke tabel in een DataFrame

for table_name, query in AdventureWorks_queries.items():
    dataframes[table_name] = pd.read_sql_query(query, Adventure_conn)

for table_name, query in Northwind_queries.items():
    dataframes[table_name] = pd.read_sql_query(query, Northwind_conn)

for table_name, query in AenC_queries.items():
    dataframes[table_name] = pd.read_sql_query(query, AenC_conn)

#als je ik elk tabel als een dataframe/ variabele wil behandelen of aanroepen moet ik dit uitvoeren.
for table_name, df in dataframes.items():
    globals()[table_name] = df


### Customer

AdventureWorks

In [43]:
# Drop duplicate columns before merging
columns_to_drop = ['rowguid', 'ModifiedDate']

# Drop duplicate columns if they exist in the DataFrames
Person = Person.drop(columns=[col for col in columns_to_drop if col in Person.columns])
BusinessEntityAddress = BusinessEntityAddress.drop(columns=[col for col in columns_to_drop if col in BusinessEntityAddress.columns])
PersonAddress = PersonAddress.drop(columns=[col for col in columns_to_drop if col in PersonAddress.columns])
StateProvince = StateProvince.drop(columns=[col for col in columns_to_drop if col in StateProvince.columns])
CountryRegion = CountryRegion.drop(columns=[col for col in columns_to_drop if col in CountryRegion.columns])
Territory = Territory.drop(columns=[col for col in columns_to_drop if col in Territory.columns])
Store = Store.drop(columns=[col for col in columns_to_drop if col in Store.columns])
PersonPhone = PersonPhone.drop(columns=[col for col in columns_to_drop if col in PersonPhone.columns])

# Drop additional duplicate columns from AW_Customer
AW_Customer = AW_Customer.drop(columns=[col for col in columns_to_drop if col in AW_Customer.columns])

# Now perform the merge operations
AW_Customer = pd.merge(AW_Customer, Person, left_on='PersonID', right_on='BusinessEntityID', suffixes=('', '_person'))
AW_Customer = pd.merge(AW_Customer, PersonPhone, on='BusinessEntityID', suffixes=('', '_phone'))
AW_Customer = pd.merge(AW_Customer, BusinessEntityAddress, on='BusinessEntityID', suffixes=('', '_business'))
AW_Customer = pd.merge(AW_Customer, PersonAddress, left_on='AddressID', right_on='AddressID', suffixes=('', '_personaddress'))
AW_Customer = pd.merge(AW_Customer, StateProvince, left_on='StateProvinceID', right_on='StateProvinceID', suffixes=('', '_stateprovince'))
AW_Customer = pd.merge(AW_Customer, CountryRegion, left_on='CountryRegionCode', right_on='CountryRegionCode', suffixes=('', '_countryregion'))
AW_Customer = pd.merge(AW_Customer, Territory, left_on='TerritoryID', right_on='TerritoryID', suffixes=('', '_Territory'))

print(AW_Customer)

       CustomerID  PersonID  StoreID  TerritoryID AccountNumber  \
0           11000   13531.0      NaN            9    AW00011000   
1           11001    5454.0      NaN            9    AW00011001   
2           11002   11269.0      NaN            9    AW00011002   
3           11003   11358.0      NaN            9    AW00011003   
4           11004   11901.0      NaN            9    AW00011004   
...           ...       ...      ...          ...           ...   
18503       29479    4191.0      NaN            7    AW00029479   
18504       29480    4472.0      NaN           10    AW00029480   
18505       29481    8168.0      NaN            8    AW00029481   
18506       29482   12570.0      NaN            7    AW00029482   
18507       29483   18384.0      NaN            7    AW00029483   

       BusinessEntityID PersonType  NameStyle Title  FirstName  ...  \
0                 13531         IN      False  None        Jon  ...   
1                  5454         IN      False  None  

Northwind

In [44]:
# First, fill NaN values in ContactName to avoid split issues
NW_Customer['ContactName'] = NW_Customer['ContactName'].fillna('')

# Split ContactName into FirstName and LastName safely
split_names = NW_Customer['ContactName'].str.split(n=1, expand=True)

# Ensure there are two columns after split
split_names.columns = ['FirstName', 'LastName']

# Handle cases where the split resulted in a single part
split_names['LastName'] = split_names['LastName'].fillna('')

# Assign the split names back to the DataFrame
NW_Customer[['FirstName', 'LastName']] = split_names

# Now drop the ContactName column
NW_Customer.drop(columns=['ContactName'], inplace=True)

# Check the DataFrame to ensure the operation was successful
print(NW_Customer.head())

  CustomerID                         CompanyName          ContactTitle  \
0      ALFKI                 Alfreds Futterkiste  Sales Representative   
1      ANATR  Ana Trujillo Emparedados y helados                 Owner   
2      ANTON             Antonio Moreno Taquería                 Owner   
3      AROUT                     Around the Horn  Sales Representative   
4      BERGS                  Berglunds snabbköp   Order Administrator   

                         Address         City Region PostalCode  Country  \
0                  Obere Str. 57       Berlin   None      12209  Germany   
1  Avda. de la Constitución 2222  México D.F.   None      05021   Mexico   
2                Mataderos  2312  México D.F.   None      05023   Mexico   
3                120 Hanover Sq.       London   None    WA1 1DP       UK   
4                Berguvsvägen  8        Luleå   None   S-958 22   Sweden   

            Phone             Fax  FirstName  LastName  
0     030-0074321     030-0076545      Ma

AenC

In [45]:
AC_Customer = pd.merge(AC_Customer, state, left_on = 'state', right_on = 'state_id')
AC_Customer = pd.merge(AC_Customer, region, on = 'region')

AC_Customer

,id,fname,lname,address,city,state,zip,phone,company_name,state_id,state_name,state_capital,country,region
0,101,Michaels,Devlin,3114 Pioneer Avenue,Rutherford,NJ,07070,2015558966,The Power Group,NJ,New Jersey,Trenton,USA,Eastern
1,102,Beth,Reiser,1033 Whippany Road,New York,NY,10154,2125558725,AMF Corp.,NY,New York,Albany,USA,Eastern
2,103,Erin,Niedringhaus,1990 Windsor Street,Paoli,PA,19301,2155556513,Darling Associates,PA,Pennsylvania,Harrisburg,USA,Eastern
3,104,Meghan,Mason,550 Dundas Street East,Knoxville,TN,37919,6155555463,P.S.C.,TN,Tennessee,Nashville,USA,Central
4,105,Laura,McCarthy,1210 Highway 36,Carmel,IN,46032,3175558437,Amo & Sons,IN,Indiana,Indianapolis,USA,Central
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,552,Janice,O'Toole,654 West Hill,Nashville,TN,37320,6155553689,Greensleeves,TN,Tennessee,Nashville,USA,Central
122,553,Stevie,Nickolas,77 Recordings Circle,Tacoma,WA,96521,5095551695,It's a Hit!,WA,Washington,Olympia,USA,Western
123,555,Philipe,Fernandez,99 Main Street,Los Angeles,CA,90205,2135554457,Quaker Fashions,CA,California,Sacramento,USA,Western
124,661,Jennifer,Stutzman,3 Back Pages Lane,Missola,IL,60505,7085556857,Stutzman Advertising,IL,Illinois,Springfield,USA,Central


In [46]:
print(NW_Customer)

   CustomerID                         CompanyName               ContactTitle  \
0       ALFKI                 Alfreds Futterkiste       Sales Representative   
1       ANATR  Ana Trujillo Emparedados y helados                      Owner   
2       ANTON             Antonio Moreno Taquería                      Owner   
3       AROUT                     Around the Horn       Sales Representative   
4       BERGS                  Berglunds snabbköp        Order Administrator   
..        ...                                 ...                        ...   
86      WARTH                      Wartian Herkku         Accounting Manager   
87      WELLI              Wellington Importadora              Sales Manager   
88      WHITC                White Clover Markets                      Owner   
89      WILMK                         Wilman Kala  Owner/Marketing Assistant   
90      WOLZA                      Wolski  Zajazd                      Owner   

                          Address      

In [47]:
# Rename columns in AC_Customer
AC_Customer.rename(columns={
    'id': 'CustomerID',
    'fname': 'FirstName',
    'lname': 'LastName',
    'phone': 'Phone',
    'region' : 'Region',
    'country': 'Country',
    'state' : 'State',
    'city' : 'City',
    'zip' : 'PostalCode',
    'address': 'Address',
    
}, inplace=True)

# Rename columns in NW_Customer
NW_Customer.rename(columns={
    'CustomerID': 'CustomerID',  # already correctly named
    'Region' : 'State'

}, inplace=True)
# Rename columns in AW_Customer
AW_Customer.rename(columns={
    'CustomerID': 'CustomerID',  # already correctly named
    'PhoneNumber': 'Phone',
    'Group' : 'Region',
    'CountryRegionCode' : 'Country',
    'StateProvinceCode' : 'State',
    'AddressLine1' : 'Address',

}, inplace=True)

# Verify the changes
print("AC_Customers after renaming:")
print(AC_Customer.head())
print("NW_Customer after renaming:")
print(NW_Customer.head())
print("AW_Customer after renaming:")
print(AW_Customer.head())


AC_Customers after renaming:
  CustomerID FirstName      LastName                 Address        City  \
0        101  Michaels        Devlin     3114 Pioneer Avenue  Rutherford   
1        102      Beth        Reiser      1033 Whippany Road    New York   
2        103      Erin  Niedringhaus     1990 Windsor Street       Paoli   
3        104    Meghan         Mason  550 Dundas Street East   Knoxville   
4        105     Laura      McCarthy         1210 Highway 36      Carmel   

  State PostalCode       Phone        company_name state_id    state_name  \
0    NJ      07070  2015558966     The Power Group       NJ    New Jersey   
1    NY      10154  2125558725           AMF Corp.       NY      New York   
2    PA      19301  2155556513  Darling Associates       PA  Pennsylvania   
3    TN      37919  6155555463              P.S.C.       TN     Tennessee   
4    IN      46032  3175558437          Amo & Sons       IN       Indiana   

  state_capital Country   Region  
0       Trenton 

Prefix

In [48]:
# Adding a prefix to EmployeeIDs for each source DataFrame
AW_Customer['CustomerID'] = AW_Customer['CustomerID'].apply(lambda x: f'AW_{x}')
NW_Customer['CustomerID'] = NW_Customer['CustomerID'].apply(lambda x: f'NW_{x}')
AC_Customer['CustomerID'] = AC_Customer['CustomerID'].apply(lambda x: f'AC_{x}')


mapping

In [49]:
country_mapping = {
    'Germany': 'DE',
    'Mexico': 'MX',
    'UK': 'GB',
    'Sweden': 'SE',
    'France': 'FR',
    'Spain': 'ES',
    'Canada': 'CA',
    'Argentina': 'AR',
    'Switzerland': 'CH',
    'Brazil': 'BR',
    'Austria': 'AT',
    'Portugal': 'PT',
    'Italy': 'IT',
    'Belgium': 'BE',
    'USA': 'US',  # Change 'USA' to 'US'
    'Ireland': 'IE',
    'Norway': 'NO',
    'Denmark': 'DK',
    'Finland': 'FI',
    'Poland': 'PL'
}

# Replace country names with abbreviations
NW_Customer['Country'] = NW_Customer['Country'].replace(country_mapping)

print(NW_Customer[['Country']])


   Country
0       DE
1       MX
2       MX
3       GB
4       SE
..     ...
86      FI
87      BR
88      US
89      FI
90      PL

[91 rows x 1 columns]


In [50]:
# Replace 'USA' with 'US' in the Country column
AC_Customer['Country'] = AC_Customer['Country'].replace('USA', 'US')

AC_Customer['Country'] = AC_Customer['Country'].replace('CAN', 'CA')

In [51]:
AC_Customer['Region'] = AC_Customer['Region'].replace({'south': 'Southern'})
AW_Customer['Region'] = AW_Customer['Region'].replace({'North America': 'Northern'})


In [52]:
required_columns_customer = [
    'CustomerID', 'FirstName', 'LastName', 'Phone', 'Region', 'Country', 'State', 'City', 
    'PostalCode', 'Address'
]

# Ensure all required columns exist in each Customer DataFrame
for col in required_columns_customer:
    if col not in AC_Customer.columns:
        AC_Customer[col] = None
    if col not in NW_Customer.columns:
        NW_Customer[col] = None
    if col not in AW_Customer.columns:
        AW_Customer[col] = None

# Select the columns in the correct order
AC_Customers = AC_Customer[required_columns_customer]
NW_Customer = NW_Customer[required_columns_customer]
AW_Customer = AW_Customer[required_columns_customer]


In [53]:
# Combine the DataFrames
Customer = pd.concat([AC_Customers, NW_Customer, AW_Customer], ignore_index=True)

# Print the data types of the combined DataFrame
print(Customer.dtypes)

# Print the first few rows to verify the concatenation
print(Customer.head())
print(Customer.info())

Customer.head


CustomerID    object
FirstName     object
LastName      object
Phone         object
Region        object
Country       object
State         object
City          object
PostalCode    object
Address       object
dtype: object
  CustomerID FirstName      LastName       Phone   Region Country State  \
0     AC_101  Michaels        Devlin  2015558966  Eastern      US    NJ   
1     AC_102      Beth        Reiser  2125558725  Eastern      US    NY   
2     AC_103      Erin  Niedringhaus  2155556513  Eastern      US    PA   
3     AC_104    Meghan         Mason  6155555463  Central      US    TN   
4     AC_105     Laura      McCarthy  3175558437  Central      US    IN   

         City PostalCode                 Address  
0  Rutherford      07070     3114 Pioneer Avenue  
1    New York      10154      1033 Whippany Road  
2       Paoli      19301     1990 Windsor Street  
3   Knoxville      37919  550 Dundas Street East  
4      Carmel      46032         1210 Highway 36  
<class 'pandas.core

<bound method NDFrame.head of       CustomerID FirstName      LastName                Phone   Region  \
0         AC_101  Michaels        Devlin           2015558966  Eastern   
1         AC_102      Beth        Reiser           2125558725  Eastern   
2         AC_103      Erin  Niedringhaus           2155556513  Eastern   
3         AC_104    Meghan         Mason           6155555463  Central   
4         AC_105     Laura      McCarthy           3175558437  Central   
...          ...       ...           ...                  ...      ...   
18720   AW_29479     Tommy          Tang  1 (11) 500 555-0136   Europe   
18721   AW_29480      Nina          Raji  1 (11) 500 555-0146   Europe   
18722   AW_29481      Ivan          Suri  1 (11) 500 555-0144   Europe   
18723   AW_29482   Clayton         Zhang  1 (11) 500 555-0137   Europe   
18724   AW_29483     Jésus       Navarro  1 (11) 500 555-0141   Europe   

      Country State              City PostalCode                    Address  
0  

# Customer segmentation

In [54]:
Customer = Customer.loc[:, ['Region', 'Country', 'State', 'City', 'PostalCode', 'Address']]

Customer

,Region,Country,State,City,PostalCode,Address
0,Eastern,US,NJ,Rutherford,07070,3114 Pioneer Avenue
1,Eastern,US,NY,New York,10154,1033 Whippany Road
2,Eastern,US,PA,Paoli,19301,1990 Windsor Street
3,Central,US,TN,Knoxville,37919,550 Dundas Street East
4,Central,US,IN,Carmel,46032,1210 Highway 36
...,...,...,...,...,...,...
18720,Europe,FR,78,Versailles,78000,"111, rue Maillard"
18721,Europe,GB,ENG,London,SW19 3RU,9 Katherine Drive
18722,Europe,DE,BY,Hof,95010,Knaackstr 4
18723,Europe,FR,17,Saint Ouen,17490,"1080, quai de Grenelle"


One encoding alle kolommen

In [55]:
dummies_dataframe = pd.get_dummies(Customer)
dummies_dataframe

,Region_Canada,Region_Central,Region_Eastern,Region_Europe,Region_Northern,Region_Pacific,Region_South,Region_Western,Country_AR,Country_AT,...,Address_Zur Lindung 40,Address_Zur Lindung 46,Address_Zur Lindung 6,Address_Zur Lindung 609,Address_Zur Lindung 7,Address_Zur Lindung 764,Address_Zur Lindung 78,Address_Zur Lindung 787,Address_ul. Filtrowa 68,Address_Åkergatan 24
0,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18720,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
18721,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
18722,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
18723,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


# Clustering model bouwen

K-means= 3

In [56]:
kmeans = KMeans(n_clusters = 3, random_state= 42) 
prediction_results = kmeans.fit_predict(dummies_dataframe)
prediction_results

array([1, 1, 1, ..., 0, 2, 2])

In [57]:
dummies_dataframe['Centrum'] = prediction_results
dummies_dataframe

,Region_Canada,Region_Central,Region_Eastern,Region_Europe,Region_Northern,Region_Pacific,Region_South,Region_Western,Country_AR,Country_AT,...,Address_Zur Lindung 46,Address_Zur Lindung 6,Address_Zur Lindung 609,Address_Zur Lindung 7,Address_Zur Lindung 764,Address_Zur Lindung 78,Address_Zur Lindung 787,Address_ul. Filtrowa 68,Address_Åkergatan 24,Centrum
0,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,1
1,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,1
2,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,1
3,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,1
4,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18720,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,2
18721,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,0
18722,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,0
18723,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,2


# Evalueren

**intra_cluster_distance \
\
Hoe kleiner hoe beter**

In [58]:
intra_cluster_distances = kmeans.inertia_
print(f'De intra_cluster_afstand is : {intra_cluster_distances}')

De intra_cluster_afstand is : 84964.56047984694


**inter_cluster_distances \
\
Hoe groter hoe beter**

In [59]:
# Retrieve the cluster centroids
centroids = kmeans.cluster_centers_

# Compute pairwise distances between centroids
distances = pairwise_distances(centroids)

# Print the inter-cluster distances for each pair of clusters
num_clusters = len(centroids)
for i in range(num_clusters):
    for j in range(i + 1, num_clusters):
        print(f'Afstand tussen centrum {i} - {j}: {distances[i, j]}')

# Calculate the average inter-cluster distance
triu_indices = np.triu_indices(len(centroids), k=1)
average_inter_cluster_distance = distances[triu_indices].mean()

print(f'The average inter-cluster distance is: {average_inter_cluster_distance}')

Afstand tussen centrum 0 - 1: 1.7563391217150335
Afstand tussen centrum 0 - 2: 1.439401484785272
Afstand tussen centrum 1 - 2: 1.8567492359857605
The average inter-cluster distance is: 1.6841632808286888


drop centrum kolom

In [60]:
# drop centrum kolom in dummies_dataframe

dummies_dataframe = dummies_dataframe.drop('Centrum', axis = 1)
dummies_dataframe

,Region_Canada,Region_Central,Region_Eastern,Region_Europe,Region_Northern,Region_Pacific,Region_South,Region_Western,Country_AR,Country_AT,...,Address_Zur Lindung 40,Address_Zur Lindung 46,Address_Zur Lindung 6,Address_Zur Lindung 609,Address_Zur Lindung 7,Address_Zur Lindung 764,Address_Zur Lindung 78,Address_Zur Lindung 787,Address_ul. Filtrowa 68,Address_Åkergatan 24
0,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18720,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
18721,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
18722,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
18723,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


# Meerdere clusters

k = 20

In [61]:
kmeans = KMeans(n_clusters = 20, random_state= 42) 
prediction_results = kmeans.fit_predict(dummies_dataframe)
prediction_results 

array([ 1,  1,  1, ..., 17,  6,  6])

In [62]:
dummies_dataframe['Centrum'] = prediction_results
dummies_dataframe

,Region_Canada,Region_Central,Region_Eastern,Region_Europe,Region_Northern,Region_Pacific,Region_South,Region_Western,Country_AR,Country_AT,...,Address_Zur Lindung 46,Address_Zur Lindung 6,Address_Zur Lindung 609,Address_Zur Lindung 7,Address_Zur Lindung 764,Address_Zur Lindung 78,Address_Zur Lindung 787,Address_ul. Filtrowa 68,Address_Åkergatan 24,Centrum
0,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,1
1,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,1
2,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,1
3,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,1
4,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18720,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,6
18721,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,3
18722,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,17
18723,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,6


**De juiste centra toewijzen aan rijen uit dataset**

In [64]:
dummies_dataframe.groupby('Centrum', as_index = False)['Centrum'].count()

,Centrum
0,406
1,3826
2,1559
3,1920
4,1584
5,2186
6,1479
7,442
8,1138
9,179


### Evalueren

**intra_cluster_distance \
\
Hoe kleiner hoe beter**

In [65]:
intra_cluster_distances = kmeans.inertia_
print(f'De intra_cluster_afstand is : {intra_cluster_distances}')

De intra_cluster_afstand is : 55170.34402611726


**inter_cluster_distances \
\
Hoe groter hoe beter**

In [66]:
# Retrieve the cluster centroids
centroids = kmeans.cluster_centers_

# Compute pairwise distances between centroids
distances = pairwise_distances(centroids)

# Print the inter-cluster distances for each pair of clusters
num_clusters = len(centroids)
for i in range(num_clusters):
    for j in range(i + 1, num_clusters):
        print(f'Afstand tussen centrum {i} - {j}: {distances[i, j]}')

# Calculate the average inter-cluster distance
triu_indices = np.triu_indices(len(centroids), k=1)
average_inter_cluster_distance = distances[triu_indices].mean()

print(f'The average inter-cluster distance is: {average_inter_cluster_distance}')

Afstand tussen centrum 0 - 1: 2.4767510015738448
Afstand tussen centrum 0 - 2: 2.517654906249414
Afstand tussen centrum 0 - 3: 2.075004030666076
Afstand tussen centrum 0 - 4: 2.512583886683393
Afstand tussen centrum 0 - 5: 2.516321788963205
Afstand tussen centrum 0 - 6: 1.8296842566065625
Afstand tussen centrum 0 - 7: 1.540124808667051
Afstand tussen centrum 0 - 8: 2.4348433321674654
Afstand tussen centrum 0 - 9: 2.688104585666886
Afstand tussen centrum 0 - 10: 2.8678727412797693
Afstand tussen centrum 0 - 11: 2.543279139784376
Afstand tussen centrum 0 - 12: 2.5388152340058125
Afstand tussen centrum 0 - 13: 2.301548329207851
Afstand tussen centrum 0 - 14: 2.868603356233276
Afstand tussen centrum 0 - 15: 2.861789548691569
Afstand tussen centrum 0 - 16: 2.7792944541099773
Afstand tussen centrum 0 - 17: 1.2642740065159863
Afstand tussen centrum 0 - 18: 2.868650345653103
Afstand tussen centrum 0 - 19: 2.8685882730464347
Afstand tussen centrum 1 - 2: 2.456355039500575
Afstand tussen centrum